In [1]:
%load_ext twisted_ipython
%autoawait twisted

In [2]:
import ipywidgets as widgets
import pandas as pd

from korbenware.config import load_config
from korbenware.logger import create_logger, PandasObserver, publisher, LEVEL_BY_NAME

In [3]:
observer = PandasObserver()
publisher.addObserver(observer)

logger = create_logger(namespace='testing')

In [4]:
logger.debug('test debug message')
logger.info('test info message')
logger.warn('test warning message')

In [5]:
observer.df

,timestamp,level,namespace,message,failure,traceback,event
0,2020-06-24 22:26:56.899242,debug,testing,test debug message,None,None,"{'log_logger': <Logger 'testing'>, 'log_level'..."
1,2020-06-24 22:26:56.907698,info,testing,test info message,None,None,"{'log_logger': <Logger 'testing'>, 'log_level'..."
2,2020-06-24 22:26:56.916026,warn,testing,test warning message,None,None,"{'log_logger': <Logger 'testing'>, 'log_level'..."


In [24]:
class LevelController:
    def __init__(self, update):
        self.update = update
        self.selectors = []
        self.values = {}
        
        for level in LEVEL_BY_NAME.keys():
            w = widgets.Checkbox(
                value=True,
                description=level
            )
            self.values[level] = True
            
            w.observe(self._checkbox_observer(level), names='value')
            
            self.selectors.append(w)
            
        self.widget = widgets.VBox(self.selectors)
        
    def _checkbox_observer(self, level):
        def observer(change):
            self.values[level] = change.new
            self.update()
        return observer
    
    def df_filter(self, df):
        f = None
        
        for level, value in self.values.items():
            if value:
                if f is None:
                    f = df['level'] == level
                else:
                    f = f | (df['level'] == level)
        
        return f

In [26]:
class ExplorerController:
    def __init__(self):
        self.observer = observer
        
        self.level_controller = LevelController(self._refresh)
        self.output = widgets.Output()
        
        self.widget = widgets.VBox([
            self.level_controller.widget,
            self.output
        ])
        
        self._refresh()
        
    def _refresh(self):
        self.output.clear_output()
        with self.output:
            filter = self.level_controller.df_filter(self.observer.df)
            if filter:
                display(self.observer.df[self.level_controller.df_filter(self.observer.df)])
            else:
                display(self.observer.df)
            
explorer_controller = ExplorerController()

explorer_controller.widget